In [2]:
import os
import shutil
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split  # Nếu không có sklearn, thay bằng manual split như dưới
import tqdm

In [9]:
# Đọc CSV
df = pd.read_csv("eyepacs_2015/trainLabels.csv")

# Manual stratified split
val_ratio = 0.2
df

,Unnamed: 0.1,Unnamed: 0,image,level
0,0,0,10_left,0
1,1,1,10_right,0
2,2,2,13_left,0
3,3,3,13_right,0
4,4,4,15_left,1
...,...,...,...,...
35103,35104,35121,44347_right,0
35104,35105,35122,44348_left,0
35105,35106,35123,44348_right,0
35106,35107,35124,44349_left,0


In [ ]:




# Đường dẫn
train_dir = 'eyepacs_2015/train_origin'
output_dir = 'eyepacs_2015'  

# Đọc CSV
df = pd.read_csv("eyepacs_2015/trainLabels.csv")


val_ratio = 0.2
df = df.sort_values('level')
df['split'] = 'train'

for level, group in df.groupby('level'):
    n_val = int(len(group) * val_ratio)
    if n_val == 0 and len(group) > 0:
        n_val = 1  # Ít nhất 1 nếu có thể
    if n_val > 0:
        val_indices = group.sample(n_val, random_state=42).index
        df.loc[val_indices, 'split'] = 'val'



# Tạo thư mục
for split in ['train', 'val']:
    for level in range(5):
        dir_path = os.path.join(output_dir, split, str(level))
        os.makedirs(dir_path, exist_ok=True)

# Xử lý ảnh
for _, row in df.iterrows():
    image_name = row['image']
    level = row['level']
    split = row['split']
    
    src_path = os.path.join(train_dir, f"{image_name}.jpeg")
    if not os.path.exists(src_path):
        print(f"File not found: {src_path}")
        continue
    
    # Open PNG and save as JPG
    img = Image.open(src_path)
    dest_dir = os.path.join(output_dir, split, str(level))
    dest_path = os.path.join(dest_dir, f"{image_name}.jpg")
    img.convert('RGB').save(dest_path, 'JPEG')
    
    print(f"Converted and moved {image_name}.png to {dest_path}")



Converted and moved 44331_right.png to eyepacs_2015/train/0/44331_right.jpg
Converted and moved 44334_left.png to eyepacs_2015/train/0/44334_left.jpg
Converted and moved 44334_right.png to eyepacs_2015/train/0/44334_right.jpg
Converted and moved 44337_right.png to eyepacs_2015/train/0/44337_right.jpg
Converted and moved 44325_left.png to eyepacs_2015/train/0/44325_left.jpg
Converted and moved 44325_right.png to eyepacs_2015/train/0/44325_right.jpg
Converted and moved 44327_left.png to eyepacs_2015/train/0/44327_left.jpg
Converted and moved 44327_right.png to eyepacs_2015/train/0/44327_right.jpg
Converted and moved 62_right.png to eyepacs_2015/train/0/62_right.jpg
Converted and moved 44343_left.png to eyepacs_2015/train/0/44343_left.jpg
Converted and moved 44343_right.png to eyepacs_2015/train/0/44343_right.jpg
Converted and moved 44347_left.png to eyepacs_2015/train/0/44347_left.jpg
Converted and moved 44347_right.png to eyepacs_2015/train/0/44347_right.jpg
Converted and moved 44330_le

In [ ]:
import os
import sys
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

# ========= CẤU HÌNH =========
ROOT_DIR = "D:\Diux\hoctap\DoAn\ddr\DR_grading"                 
CSV_PATH = "D:\Diux\hoctap\DoAn\ddr\DR_grading.csv"
SPLIT_RATIOS = (0.7, 0.2, 0.1)            # train, val, test
RANDOM_STATE = 42
CONVERT_TO_JPEG = True                   

# ========= ĐỌC CSV & XÁC ĐỊNH CỘT =========
df = pd.read_csv(CSV_PATH)


ID_CANDIDATES = ["id_code", "image", "filename", "file", "image_id"]
LABEL_CANDIDATES = ["level", "diagnosis", "label", "grade"]

id_col = next((c for c in ID_CANDIDATES if c in df.columns), None)
label_col = next((c for c in LABEL_CANDIDATES if c in df.columns), None)
if id_col is None or label_col is None:
    print("Không tìm thấy cột id/nhãn trong CSV. Các cột hiện có:", df.columns.tolist())
    sys.exit(1)

# Đảm bảo kiểu dữ liệu chuẩn
df[id_col] = df[id_col].astype(str)
df[label_col] = df[label_col].astype(int)

# ========= CHIA TẬP LINH HOẠT THEO STRATIFY =========
train_ratio, val_ratio, test_ratio = SPLIT_RATIOS
assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Tổng tỷ lệ phải bằng 1.0"

# Bước 1: Tách tập train
train_df, remain_df = train_test_split(
    df, test_size=(val_ratio + test_ratio),
    stratify=df[label_col], random_state=RANDOM_STATE
)

# Bước 2: Tách tập val và test từ phần còn lại
val_relative = val_ratio / (val_ratio + test_ratio)
val_df, test_df = train_test_split(
    remain_df, test_size=(1 - val_relative),
    stratify=remain_df[label_col], random_state=RANDOM_STATE
)
splits = {
    "train": train_df.reset_index(drop=True),
    "val":   val_df.reset_index(drop=True),
    "test":  test_df.reset_index(drop=True),
}

print({k: len(v) for k, v in splits.items()})
print("Phân bố lớp (train):\n", train_df[label_col].value_counts().sort_index())
print("Phân bố lớp (val):\n",   val_df[label_col].value_counts().sort_index())
print("Phân bố lớp (test):\n",  test_df[label_col].value_counts().sort_index())

# ========= TẠO THƯ MỤC ĐÍCH =========
for split in ["train", "val", "test"]:
    for lvl in range(5):
        os.makedirs(os.path.join(ROOT_DIR, split, str(lvl)), exist_ok=True)


POSSIBLE_EXTS = [".png", ".jpg", ".jpeg", ".JPG", ".JPEG", ".PNG", ".tif", ".tiff"]

def find_image_path(root_dir, image_id):
    """
    Tìm ảnh trong ROOT_DIR với các phần mở rộng phổ biến.
    image_id có thể đã có/không có đuôi; hàm thử lần lượt.
    """
    # Nếu image_id đã có đuôi, thử trực tiếp trước
    base, ext = os.path.splitext(image_id)
    candidates = []
    if ext:
        candidates.append(os.path.join(root_dir, image_id))
        image_id = base  
    # Thử các đuôi chuẩn
    for e in POSSIBLE_EXTS:
        candidates.append(os.path.join(root_dir, f"{image_id}{e}"))

    for p in candidates:
        if os.path.exists(p) and os.path.isfile(p):
            return p
    return None

# ========= SAO CHÉP/CHUẨN HOÁ ẢNH THEO THƯ MỤC =========
not_found = 0
failed = 0
rows_out = []

for split_name, split_df in splits.items():
    print(f"\nĐang xử lý split = {split_name} ({len(split_df)} ảnh)")
    for _, row in tqdm(split_df.iterrows(), total=len(split_df)):
        image_id = str(row[id_col])
        level = int(row[label_col])

        src_path = find_image_path(ROOT_DIR, image_id)
        if src_path is None:
            not_found += 1
            
            print(f"Không tìm thấy ảnh cho id = {image_id}")
            continue

        dest_dir = os.path.join(ROOT_DIR, split_name, str(level))
        dest_path = os.path.join(dest_dir, f"{os.path.splitext(image_id)[0]}.jpg")

        try:
            if CONVERT_TO_JPEG:
                with Image.open(src_path) as img:
                    img = img.convert("RGB")
                    
                    img.save(dest_path, "JPEG", quality=100, optimize=True)
            else:
             
                with Image.open(src_path) as img:
                    img = img.convert("RGB")
                    img.save(dest_path, "JPEG", quality=100, optimize=True)

            rows_out.append({
                "id": image_id,
                "label": level,
                "split": split_name,
                "src": os.path.relpath(src_path, ROOT_DIR),
                "dst": os.path.relpath(dest_path, ROOT_DIR)
            })
        except Exception as e:
            failed += 1
            print(f"[ERR] {image_id} -> {dest_path}: {e}")

print(f"\nHoàn tất. Miss={not_found}, Fail={failed}")

# Lưu lại file tổng hợp 
out_csv = os.path.join(ROOT_DIR, "splits_index.csv")
pd.DataFrame(rows_out).to_csv(out_csv, index=False)
print(f"Đã lưu chỉ mục split: {out_csv}")


<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
C:\Users\tran1\AppData\Local\Temp\ipykernel_11500\3110721867.py:9: SyntaxWarning: invalid escape sequence '\D'
  ROOT_DIR = "D:\Diux\hoctap\DoAn\ddr\DR_grading"                 # Thư mục gốc: chứa ảnh gốc + file CSV
C:\Users\tran1\AppData\Local\Temp\ipykernel_11500\3110721867.py:10: SyntaxWarning: invalid escape sequence '\D'
  CSV_PATH = "D:\Diux\hoctap\DoAn\ddr\DR_grading.csv"


{'train': 8765, 'val': 2504, 'test': 1253}
Phân bố lớp (train):
 diagnosis
0    4386
1     441
2    3134
3     165
4     639
Name: count, dtype: int64
Phân bố lớp (val):
 diagnosis
0    1253
1     126
2     895
3      47
4     183
Name: count, dtype: int64
Phân bố lớp (test):
 diagnosis
0    627
1     63
2    448
3     24
4     91
Name: count, dtype: int64

Đang xử lý split = train (8765 ảnh)


100%|██████████| 8765/8765 [04:38<00:00, 31.50it/s]



Đang xử lý split = val (2504 ảnh)


100%|██████████| 2504/2504 [02:04<00:00, 20.13it/s]



Đang xử lý split = test (1253 ảnh)


100%|██████████| 1253/1253 [01:04<00:00, 19.34it/s]



Hoàn tất. Miss=0, Fail=0
Đã lưu chỉ mục split: D:\Diux\hoctap\DoAn\ddr\DR_grading\splits_index.csv
